# MAD-X Simulations

This section is based on the **BL2_Triplet_TDC_full** (Beamline 2, transverse deflector cavity in operation) ATF example.

**Hint**: See the [List of predefined simulations in Sirepo](../simulations.rst) for examples of identifiers for different beamlines.

### Setup and scan model before changing parameters

In [ ]:
from sirepo_bluesky import prepare_re_env

%run -i $prepare_re_env.__file__ -e flyer

import matplotlib.pyplot as plt

from sirepo_bluesky.common.sirepo_client import SirepoClient
from sirepo_bluesky.madx.madx_flyer import MADXFlyer
from sirepo_bluesky.common.create_classes import create_classes

connection = SirepoClient("http://localhost:8000")

data, schema = connection.auth("madx", "00000002")
classes, objects = create_classes(
    connection=connection,
    extra_model_fields=["rpnVariables", "commands"],
)
globals().update(**objects)

madx_flyer = MADXFlyer(
    connection=connection,
    root_dir="/tmp/sirepo-bluesky-data",
    report="elementAnimation250-20",
)

(uid1,) = RE(bp.fly([madx_flyer]))
hdr1 = db[uid1]
tbl1 = hdr1.table(stream_name="madx_flyer", fill=True)
print(tbl1)

### Scan again after changing parameters

In [ ]:
def madx_plan(element=match8, parameter=ihq1, value=1.0):
    yield from bps.mv(element.deltap, value)
    yield from bps.mv(parameter.value, value)
    return (yield from bp.fly([madx_flyer]))


(uid2,) = RE(madx_plan())
hdr2 = db[uid2]
tbl2 = hdr2.table(stream_name="madx_flyer", fill=True)
print(tbl2)

### Get the data via databroker API and plot

In [ ]:
# Before changing parameters
s1 = tbl1["madx_flyer_S"]
betx1 = tbl1["madx_flyer_BETX"]
bety1 = tbl1["madx_flyer_BETY"]

plt.figure()
plt.plot(s1, betx1, label="betx")
plt.plot(s1, bety1, label="bety")
plt.xlabel("s [m]")
plt.ylabel("betx, bety [m]")
plt.title("Before")
plt.grid()
plt.legend()

In [ ]:
# After changing parameters
s2 = tbl2["madx_flyer_S"]
betx2 = tbl2["madx_flyer_BETX"]
bety2 = tbl2["madx_flyer_BETY"]

plt.figure()
plt.plot(s2, betx2, label="betx")
plt.plot(s2, bety2, label="bety")
plt.xlabel("s [m]")
plt.ylabel("betx, bety [m]")
plt.title("After")
plt.grid()
plt.legend()

### Get the data from the raw TFS file and plot

In [ ]:
import os
import tfs

for name, doc in hdr2.documents():
    if name == "resource":
        resource_doc = doc
        print(resource_doc)

filename = os.path.join(resource_doc["root"], resource_doc["resource_path"])
df = tfs.read(filename)

In [ ]:
df

In [ ]:
plt.figure()
plt.plot(df["S"], df["BETX"], label="betx")
plt.plot(df["S"], df["BETY"], label="bety")
plt.xlabel("s [m]")
plt.ylabel("betx, bety [m]")
plt.title("After")
plt.grid()
plt.legend()

## Screenshots from Sirepo Website

### Original model, before changing parameters

![image.png](../../../images/madx_before_twiss3_file.png)

### After changing parameters

![image2.png](../../../images/madx_after_twiss3_file.png)
